creating db, profiles for consumption, generation and wholesale market price

In [1]:
### create profiles for consumption, generation and wholesale market price

import pandas as pd
# import datetime
import pickle

# Get all holiday dates in 2019 in Vermont
import holidays
vt_holidays = holidays.US(subdiv='VT', years=2019)

# Read the excel file
file_path = 'profiles.xlsx'
sheet_names = ['House 1', 'House 2', 'House 3', 'House 4']

# Create an empty dataframe with 15-minute intervals for the whole year 2019
start_date = '2019-01-01 00:00:00'
end_date = '2019-12-30 23:45:00'
date_range = pd.date_range(start=start_date, end=end_date, freq='15T')
combined_df = pd.DataFrame(date_range, columns=['Msrmt Date Time'])

# Iterate through each sheet (house) in the Excel file
for sheet_name in sheet_names:
    # Load data from the sheet
    house_df = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl', usecols=['kWH Consumed', 'Msrmt Date Time'])

    # Convert 'Msrmt Date Time' column to datetime
    house_df['Msrmt Date Time'] = pd.to_datetime(house_df['Msrmt Date Time'])

    # Rename 'kWH Consumed' column to match the house name
    house_df.rename(columns={'kWH Consumed': f'{sheet_name}'}, inplace=True)

    # Merge the data with the main dataframe
    combined_df = pd.merge(combined_df, house_df, on='Msrmt Date Time', how='left')

    # Fill missing values with data from the previous day
    combined_df[f'{sheet_name}'] = combined_df[f'{sheet_name}'].fillna(method='ffill', axis=0)

# Adding PV generation

house_df = pd.read_excel(file_path, sheet_name='PV', engine='openpyxl', usecols=['kWH Generated', 'Msrmt Date Time'])
house_df['Msrmt Date Time'] = pd.to_datetime(house_df['Msrmt Date Time'])
house_df = house_df.groupby('Msrmt Date Time').sum().reset_index()
house_df.rename(columns={'kWH Consumed': f'{sheet_name}'}, inplace=True)
combined_df = pd.merge(combined_df, house_df, on='Msrmt Date Time', how='left')
combined_df[f'{sheet_name}'] = combined_df[f'{sheet_name}'].fillna(method='ffill', axis=0)

# Adding Price
price_df = pd.read_excel(file_path, sheet_name='Price', engine='openpyxl', usecols=['Date', 'Hr_End', 'DA_LMP'])

# Combine the 'Date' and 'Hr_End' columns into a single datetime column and set it as the index
price_df['Datetime'] = pd.to_datetime(price_df['Date']) + pd.to_timedelta(price_df['Hr_End'] - 1, unit='h')
price_df.set_index('Datetime', inplace=True)

# Resample the data to have a 15-minute frequency by repeating the price for all steps in an hour
resampled_price_df = price_df['DA_LMP'].resample('15T').ffill()

# Reset the index to make 'Datetime' a regular column
resampled_price_df = resampled_price_df.reset_index()

# Merge the resampled price data with the current dataframe
combined_df = pd.merge(combined_df, resampled_price_df, left_on='Msrmt Date Time', right_on='Datetime', how='left')
combined_df['DA_LMP'] /= 1000
# Drop the 'Datetime' column, as it is redundant
combined_df.drop('Datetime', axis=1, inplace=True)

# Reset the index and save the final dataframe to a new Excel file
combined_df.reset_index(drop=True, inplace=True)
# combined_df.to_excel('combined_data.xlsx', index=False)
combined_df.rename(columns={'Msrmt Date Time': 'DateTime', 'DA_LMP': 'Price', 'kWH Generated': 'PV'}, inplace=True)
def add_TOU_and_EV_availability(row):
    tou_value = 0.12206
    ev_availability = 1

    # Check if it's a weekday (Monday=0, Sunday=6)
    if row['DateTime'].weekday() < 5:
        # Check if the time is between 1:00 pm and 9:00 pm
        if row['DateTime'].time() >= pd.Timestamp("13:00:00").time() and row['DateTime'].time() < pd.Timestamp("21:00:00").time():
            tou_value = 0.28638
            # Check if the date is a holiday
    if row['DateTime'].date() in vt_holidays:
        pass
    # Check if the time is between 8:45 am and 5:15 pm
    elif row['DateTime'].time() >= pd.Timestamp("08:45:00").time() and row['DateTime'].time() < pd.Timestamp("17:15:00").time():
            ev_availability = 0

    return tou_value, ev_availability

# Apply the custom function to each row and create new columns
combined_df['TOU'], combined_df['EV_availability'] = zip(*combined_df.apply(add_TOU_and_EV_availability, axis=1))
combined_df['PV'] = combined_df['PV'].fillna(method='ffill')

# df = db['ld']

# # Add a new column with the month for each row
# df['Month'] = df['DateTime'].dt.month

# # Add a new column 'Month_Change' which is True if the month changes compared to the previous row
# df['Month_Change'] = df['Month'] != df['Month'].shift(1)

# # Find the rows where the month changes
# month_change_rows = df[df['Month_Change']]

# tk = month_change_rows.index.to_list()

db={}
db['ld'] = combined_df
db['scenarios'] = {
    "s0" : {'B_max': 0,    'B_min': 0,  'U_c': 0,    'U_d': 0,    'E_max': 0,  'E_min': 0,  'use_pv': False, 'price': 0},
    "s1" : {'B_max': 40.5, 'B_min': 0,  'U_c': 3.75, 'U_d': 3.75, 'E_max': 0,  'E_min': 0,  'use_pv': False, 'price': 27300},
    "s2" : {'B_max': 27,   'B_min': 0,  'U_c': 2.5,  'U_d': 2.5,  'E_max': 0,  'E_min': 0,  'use_pv': True,  'price': 35125},
    "s3" : {'B_max': 0,    'B_min': 0,  'U_c': 1.9,  'U_d': 1.9,  'E_max': 60, 'E_min': 24, 'use_pv': False, 'price': 4000},
    "s4" : {'B_max': 0,    'B_min': 0,  'U_c': 1.9,  'U_d': 1.9,  'E_max': 60, 'E_min': 24, 'use_pv': True,  'price': 17145},
    "s5" : {'B_max': 10,   'B_min': 0,  'U_c': 1.9,  'U_d': 1.9,  'E_max': 60, 'E_min': 24, 'use_pv': True,  'price': 23724}
}
db['T'] = db['ld'].shape[0]
db['Usage'] = 10
db['eta_c'] = 0.95
db['eta_d'] = 0.95
db['tk'] = [0, 2976, 5664, 8640, 11520, 14496, 17376, 20352, 23328, 26208, 29184, 32124, 35099]
with open('db.pkl', 'wb') as file:
    pickle.dump(db, file, protocol=pickle.HIGHEST_PROTOCOL)
combined_df.head(5)

,DateTime,House 1,House 2,House 3,House 4,PV,Price,TOU,EV_availability
0,2019-01-01 00:00:00,0.55,0.42,0.42,0.31,0.0,0.02493,0.12206,1
1,2019-01-01 00:15:00,0.21,0.46,0.43,0.28,0.0,0.02493,0.12206,1
2,2019-01-01 00:30:00,0.40,0.40,0.50,0.33,0.0,0.02493,0.12206,1
3,2019-01-01 00:45:00,0.51,0.41,0.48,0.24,0.0,0.02493,0.12206,1
4,2019-01-01 01:00:00,0.15,0.42,0.45,0.23,0.0,0.02023,0.12206,1


importing libraries and modelling

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import gurobipy as gp
from gurobipy import GRB
from tqdm import tqdm
import pickle
import numpy_financial as npf
import gc
import plotly.express as px
pd.options.display.max_rows = 500
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.set_option("display.precision", 2)

In [2]:
def model(db, hn, s, p, sell):
    #hn: house number, s: scenario, p: rate, sell: if selling back is allowed; if p == gmp, it means gmp is controlling
    T = db['T']
    pev = 0.16859
    if p == 'flat':
        P_CHARGE = 0.492/24/4
        P = np.ones(T) * 0.16859
    elif p == 'Time-of-Use':
        P_CHARGE = 0.651/24/4
        P = db['ld'].TOU
    elif p == 'RTLMP':
        P_CHARGE = 0
        P = db['ld'].Price
    elif p == 'TOU&RTLMP':
        P_CHARGE = 0.651/24/4
        P = db['ld'].TOU
    elif p == 'gmp':
        P_CHARGE = 0
        P = db['ld'].Price
    else:
        P_CHARGE = 0.492/24/4
        P = np.ones(T) * 0.16859
        P_EV = db['ld'].TOU
    if sell == True:
        if p == 'TOU&RTLMP':
            p_sell = db['ld'].Price
        else:
            p_sell = P
    else :
        p_sell = np.zeros(T)
        
    L = db['ld'][hn]
    if s in ['s2', 's4', 's5']:
        R = db['ld'].PV
    else:
        R = np.zeros(T)
    if s in ['s3', 's4', 's5']:
        A = db['ld']['EV_availability']
        V = (1-A) * 10 / (9*4+2)
    else:
        V = np.zeros(T)
        A = np.zeros(T)
    tk = db['tk']
    # Model formulation
    m = gp.Model('home')
    m.setParam('OutputFlag', 0)
    # Decision Vars
    x_GL   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^GL')
    x_GB   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^GB')
    x_GE   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^GE')
    x_RL   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^RL')
    x_RB   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^RB')
    x_RE   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^RE')
    x_RG   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^RG')
    x_BL   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^BL')
    x_EL   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^EL')
    x_BG   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^BG')
    x_EG   = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='x^EG')
    x_PEAK = m.addVars(12, lb=0.0, vtype=GRB.CONTINUOUS, name='x^PEAK')
    b_E    = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='b^E')
    b_B    = m.addVars(T, lb=0.0, vtype=GRB.CONTINUOUS, name='b^B')
    # Constraints
    m.addConstrs((x_GL[t] + db['eta_d'] * x_BL[t] + db['eta_d'] * x_EL[t] + x_RL[t] == L[t] for t in range(T)), name='c_load')
    m.addConstrs((x_BL[t] + x_BG[t] <= b_B[t] for t in range(T)), name='c_bat_state')
    m.addConstrs((x_EL[t] + x_EG[t] <= b_E[t] * A[t] for t in range(T)), name='c_ev_state')
    m.addConstrs((x_RE[t] + x_GE[t] <= db['scenarios'][s]['E_max'] * A[t] for t in range(T)), name='c_ev_2_state')
    m.addConstrs((x_BL[t] + x_BG[t] + x_EL[t] + x_EG[t] <= db['scenarios'][s]['U_d'] for t in range(T)), name='c_dch_max')
    m.addConstrs((x_GB[t] + x_RB[t] + x_GE[t] + x_RE[t] <= db['scenarios'][s]['U_c'] for t in range(T)), name='c_ch_max')
    m.addConstrs((b_B[t] <= db['scenarios'][s]['B_max'] for t in range(T)), name='c_b_B_max')
    m.addConstrs((b_B[t] >= db['scenarios'][s]['B_min'] for t in range(T)), name='c_b_B_min')
    m.addConstrs((b_E[t] <= db['scenarios'][s]['E_max'] for t in range(T)), name='c_b_E_max')
    m.addConstrs((b_E[t] >= db['scenarios'][s]['E_min'] for t in range(T)), name='c_b_E_min')
    m.addConstr((b_B[0] == 0 ), name='c_b_B_0')
    m.addConstr((b_E[0] == db['scenarios'][s]['E_min'] ), name='c_b_E_0')
    m.addConstrs((x_RL[t] + x_RB[t] + x_RE[t] + x_RG[t] <= R[t] for t in range(T)), name='c_gen_cap')
    m.addConstrs((b_B[t] + db['eta_c'] * (x_GB[t] + x_RB[t]) - (x_BL[t] + x_BG[t]) == b_B[t+1] for t in range(T-1)), name='c_next_state_b_B')
    m.addConstrs((b_E[t] + db['eta_c'] * (x_GE[t] + x_RE[t]) - (x_EL[t] + x_EG[t]) - (V[t] / db['eta_d']) == b_E[t+1] for t in range(T-1)), name='c_next_state_b_E')
    for k in range(12):
        for t in range(tk[k]+1,tk[k+1]+1):
            m.addConstr((x_PEAK[k] >= x_GB[t] +  x_GE[t] + x_GL[t]), name='maxconstr')
    if sell == False:
        m.addConstrs((x_BG[t] + x_RG[t] + x_EG[t] == 0 for t in range(T)), name='c_trade')
    # Objective
    if p == 'gmp':
        m.setObjective(gp.quicksum(P[t] * (1.0001 * (x_GB[t] + x_GE[t] + x_GL[t])) - p_sell[t] * (0.9999 * (x_RG[t] + db['eta_d'] * (x_BG[t]+ x_EG[t])))
                                for t in range(T)) +gp.quicksum((1.35*5+9) * 4 * x_PEAK[k] for k in range(12)), GRB.MINIMIZE)
    elif p == 'Time-of-Use EV & flat':
        m.setObjective(gp.quicksum(P_CHARGE + P_EV[t] * 1.0001 * x_GE[t] + P[t] * 1.0001 * (x_GB[t] + x_GL[t]) - 0.9999 * p_sell[t] * (x_RG[t] + db['eta_d'] * (x_BG[t] + x_EG[t])) for t in range(T)), GRB.MINIMIZE)
    else:
        m.setObjective(gp.quicksum(P_CHARGE + P[t] * 1.0001 * (x_GB[t] + x_GE[t] + x_GL[t]) - p_sell[t] * 0.999 * (x_RG[t] + db['eta_d'] * (x_BG[t] + x_EG[t])) for t in range(T)), GRB.MINIMIZE)
    # Optimize
    m.optimize()
    if p == 'Time-of-Use EV & flat':
        gmp = np.sum([(P_EV[t] - db['ld'].Price[t]) * x_GE[t].x + (P[t] - db['ld'].Price[t]) * (x_GB[t].x + x_GL[t].x) - (p_sell[t] - db['ld'].Price[t]) * (x_RG[t].x + db['eta_d'] * (x_BG[t].x + x_EG[t].x)) for t in range(T)])
    elif p == 'gmp':
        gmp = 0
    else:
        gmp = np.sum([(P[t] - db['ld'].Price[t]) * (x_GB[t].x + x_GE[t].x + x_GL[t].x) - (p_sell[t] - db['ld'].Price[t]) * (x_RG[t].x + db['eta_d'] * (x_BG[t].x + x_EG[t].x)) for t in range(T)])
    peaks = [x_PEAK[t].x for t in range(12)]
    from_grid = np.sum([x_GB[t].x + x_GE[t].x + x_GL[t].x for t in range(T)])
    to_grid = np.sum([x_RG[t].x + db['eta_d'] * (x_BG[t].x + x_EG[t].x) for t in range(T)])
    objval = m.objVal
    m.dispose()
    del m
    gc.collect()
    return objval-V.sum()*pev, peaks, gmp, from_grid, to_grid


## running model for different configurations

In [2]:
with open('db.pkl', 'rb') as file:
    db = pickle.load(file)
df = db['df']
df_s1 = db['df_s1']
df_s2 = db['df_s2']

### House characteristics

In [3]:
with open('db.pkl', 'rb') as file:
    db = pickle.load(file)
ddf = db['ld']
ddf[["House 1",	"House 2",	"House 3",	"House 4"]] *= 4
ddf['DateTime'] = pd.to_datetime(ddf['DateTime'])
first_week_df = ddf[ddf['DateTime'].dt.strftime('%W') == '00']
# Create the line plot
fig = px.line(first_week_df, x='DateTime', y=['House 1', 'House 2', 'House 3', 'House 4'])
fig.update_layout(
    legend_title='House Number'
)

# Customize the y-axis label
fig.update_yaxes(title_text='Consumption (kWh)')
# Show the plot
fig.show()

In [50]:
des = 4*df[['House 1', 'House 2', 'House 3', 'House 4']].describe().drop('count')
des.loc['sum', :] = [df['House 1'].sum(),df['House 2'].sum(),df['House 3'].sum(),df['House 4'].sum()]
des.T

,mean,std,min,25%,50%,75%,max,sum
House 1,0.99,0.89,0.00,0.44,0.64,1.20,9.16,8709.21
House 2,0.78,1.06,0.00,0.24,0.40,0.72,8.08,6861.55
House 3,1.61,1.60,0.00,0.60,1.04,2.04,13.12,14147.57
House 4,1.06,1.21,0.00,0.32,0.60,1.20,9.72,9345.24


In [51]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Box(y=4*df['House 1'],name='House 1',boxmean='sd', boxpoints = False))
fig.add_trace(go.Box(y=4*df['House 2'],name='House 2',boxmean='sd', boxpoints = False))
fig.add_trace(go.Box(y=4*df['House 3'],name='House 3',boxmean='sd', boxpoints = False))
fig.add_trace(go.Box(y=4*df['House 4'],name='House 4',boxmean='sd', boxpoints = False))
fig.update_layout(yaxis_title = 'kWh')
fig.show()

In [12]:
import pandas as pd
import pickle
from tqdm import tqdm

data = {'agent': [], 'house_number': [], 'scenario': [], 'rate': [], 'selling_back': [], 'annual_cost': [], 'saving': [], 'gmp_trade_cost': [], 'network_cost': [], 'npv': [], 'irr': [], 'load_from_grid': [], 'load_to_grid': [], 'max_peak_load': [], 'peaks': []}
df = pd.DataFrame(data)

with open('db.pkl', 'rb') as file:
    db = pickle.load(file)
# N: life time 
N= 15
# ir: interest rate
ir = 0.0619
house_numbers = ['House 1', 'House 2', 'House 3', 'House 4']
scenarios = ['s0', 's1', 's2', 's3', 's4', 's5']
rates = ['flat', 'Time-of-Use', 'Time-of-Use EV & flat', 'RTLMP', 'TOU&RTLMP', 'gmp']
selling_options = [True, False]

total_iterations = len(house_numbers) * len(scenarios) * len(rates) * len(selling_options)
progress_bar = tqdm(total=total_iterations, desc="Processing", ncols=100)
cc = 0
for hn in house_numbers:
    for s in scenarios:
        for p in rates:
            for sell in selling_options:
                new_row = {'house_number': hn, 'scenario': s, 'rate': p, 'selling_back': sell}
                new_row['agent'] = 'GMP' if p == 'gmp' else 'PRO'
                objval, peaks, gmp_trade_revenue, from_grid, to_grid = model(db=db, hn=hn, s=s, p=p, sell=sell)
                new_row['network_cost'] = (1.35*5+9) * 4 * np.sum(peaks)
                new_row['annual_cost'] = objval
                new_row['load_from_grid'] = from_grid
                new_row['load_to_grid'] = to_grid
                new_row['gmp_trade_cost'] = gmp_trade_revenue
                new_row['max_peak_load'] = 4 * np.max(peaks)
                new_row['peaks'] = str(np.round(peaks,2))
                if s == 's0':
                    saving = 0
                else:
                    sq_cost = df.query("rate == @p and house_number == @hn and selling_back == @sell and scenario == 's0'")['annual_cost'].values[0]
                    saving = sq_cost - objval
                new_row['saving'] = saving
                returns = [-db['scenarios'][s]['price']]+N*[saving]
                new_row['npv'] = npf.npv(ir, returns)
                new_row['irr'] = 100 * npf.irr(returns)                
                new_row_df = pd.DataFrame([new_row])
                df = pd.concat([df, new_row_df], ignore_index=True)
                # cc += 1
                # print(cc)
                progress_bar.update(1)
df.loc[0, 'selling_back'] = True
progress_bar.close()
df_backup = df.copy()
db['df'] = df

Processing: 100%|███████████████████████████████████████████████| 288/288 [2:00:17<00:00, 25.06s/it]


In [21]:
df_gmp = df.query("house_number == 'House 3' and rate == 'gmp' and selling_back == True").copy()
df_gmp['net_trade'] = df_gmp['annual_cost'] - df_gmp['network_cost']
columns_to_print = ['scenario', 'net_trade', 'network_cost', 'annual_cost', 'saving', 'npv', 'irr', 'max_peak_load', 'load_from_grid', 'load_to_grid']
# columns_to_print = ['net_trade', 'network_cost', 'annual_cost', 'saving', 'npv', 'irr', 'max_peak_load', 'load_from_grid', 'load_to_grid']
df_gmp = df_gmp[columns_to_print]
df_gmp

,scenario,net_trade,network_cost,annual_cost,saving,npv,irr,max_peak_load,load_from_grid,load_to_grid
154,s0,505.34,2050.02,2555.36,0.00,0.00,NaN,13.12,14147.57,0.00
166,s1,421.60,486.48,908.08,1647.28,-11497.94,-1.22,3.75,19745.19,4768.63
178,s2,60.51,647.42,707.92,1847.44,-17402.90,-2.83,5.07,17636.75,13860.35
190,s3,-78.75,1820.70,1741.95,813.41,3802.85,18.80,11.64,26057.46,7351.90
202,s4,-399.09,1604.61,1205.52,1349.84,-4196.27,2.15,11.60,22595.84,15459.30
214,s5,-380.28,772.19,391.91,2163.45,-2970.48,4.20,6.33,20069.33,12954.01


In [77]:
df_pro = df.query("house_number == 'House 4' and agent == 'PRO' and selling_back == True").copy()
# df_pro['net_trade'] = df_pro['annual_cost'] - df_pro['network_cost']
columns_to_print = ['rate', 'scenario', 'annual_cost', 'network_cost', 'saving', 'npv', 'irr', 'max_peak_load', 'load_from_grid', 'load_to_grid']
# columns_to_print = ['net_trade', 'network_cost', 'annual_cost', 'saving', 'npv', 'irr', 'max_peak_load', 'load_from_grid', 'load_to_grid']
df_pro = df_pro[columns_to_print]
rate_order = ['flat', 'Time-of-Use', 'Time-of-Use EV & flat', 'RTLMP', 'TOU&RTLMP']
scenario_order = ['s0', 's1', 's2', 's3', 's4', 's5']

df_pro['rate'] = pd.Categorical(df_pro['rate'], categories=rate_order, ordered=True)
df_pro['scenario'] = pd.Categorical(df_pro['scenario'], categories=scenario_order, ordered=True)

df_pro = df_pro.sort_values(['rate', 'scenario'])

df_pro2 = df.query("house_number == 'House 4' and agent == 'PRO' and selling_back == False").copy()
# df_pro2['net_trade'] = df_pro2['annual_cost'] - df_pro2['network_cost']
columns_to_print = ['rate', 'scenario', 'npv', 'network_cost']
# columns_to_print = ['net_trade', 'network_cost', 'annual_cost', 'saving', 'npv', 'irr', 'max_peak_load', 'load_from_grid', 'load_to_grid']
df_pro2 = df_pro2[columns_to_print]
rate_order = ['flat', 'Time-of-Use', 'Time-of-Use EV & flat', 'RTLMP', 'TOU&RTLMP']
scenario_order = ['s0', 's1', 's2', 's3', 's4', 's5']

df_pro2['rate'] = pd.Categorical(df_pro2['rate'], categories=rate_order, ordered=True)
df_pro2['scenario'] = pd.Categorical(df_pro2['scenario'], categories=scenario_order, ordered=True)

df_pro2 = df_pro2.sort_values(['rate', 'scenario'])
df_pro2['rate'] = df_pro2['rate'].replace({
    'flat': 'Flat',
    'Time-of-Use': 'TOU',
    'Time-of-Use EV & flat': 'TOU EV & Flat'
})
df_pro['npv_2'] = df_pro2['npv'].to_numpy()
df_pro['network_cost_2'] = df_pro2['network_cost'].to_numpy()
print(df_pro.round())

                      rate scenario  annual_cost  network_cost  saving       npv    irr  max_peak_load  load_from_grid  load_to_grid     npv_2  network_cost_2
216                   flat       s0      1756.00       1631.00    0.00      0.00    NaN          10.00         9345.00          0.00      0.00         1631.00
228                   flat       s1      1756.00       1631.00    0.00 -27300.00    NaN          10.00         9345.00          0.00 -27300.00         1631.00
240                   flat       s2      -189.00       1555.00 1945.00 -16468.00  -2.00          10.00         6735.00       8935.00 -25144.00         1130.00
252                   flat       s3      1813.00       2662.00  -57.00  -4549.00    NaN          17.00        12845.00          2.00  -4552.00         2672.00
264                   flat       s4      -132.00       2457.00 1888.00    965.00   7.00          17.00         9063.00       7765.00 -11528.00         2445.00
276                   flat       s5      -132.

### resluts when selling back is allowed

In [31]:
# Define color dictionary
color_dict = {
    "s0": "#636EFA",
    "s1": "#EF553B",
    "s2": "#00CC96",
    "s3": "#AB63FA",
    "s4": "#FFA15A",
    "s5": "#19D3F3"
}

legend = {
    "s0": "s0: SQ",
    "s1": "s1: Bat",
    "s2": "s2: Bat+PV",
    "s3": "s3: EV",
    "s4": "s4: EV+PV",
    "s5": "s5: Bat+EV+PV"
}


# for p in rates:
for p in ['flat', 'Time-of-Use', 'Time-of-Use EV & flat', 'RTLMP', 'TOU&RTLMP', 'gmp']:
    df1 = df.query('selling_back == True and rate == @p')
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # npv bar plot
    for scenario in df1['scenario'].unique():
        df2 = df1[df1['scenario'] == scenario]
        fig.add_trace(go.Bar(x=df2['house_number'], y=df2['npv'], name=f'NPV-{legend[scenario]}', marker_color=color_dict[scenario]), secondary_y=False)

    # irr line plot
    for scenario in df1['scenario'].unique():
        df2 = df1[df1['scenario'] == scenario]
        fig.add_trace(go.Scatter(x=df2['house_number'], y=df2['irr'], mode='lines+markers', name=f'IRR-{legend[scenario]}', line=dict(color=color_dict[scenario])), secondary_y=True)

    # Set titles and labels
    fig.update_layout(title_text=f"NPV & IRR, rate: {p}")
    fig.update_xaxes(title_text="House Number")
    fig.update_yaxes(title_text="NPV", secondary_y=False)
    fig.update_yaxes(title_text="IRR", secondary_y=True)
    
    # Show the figure
    fig.show()


### NPV and IRR when selling back is not allowed

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define color dictionary
color_dict = {
    "s0": "#636EFA",
    "s1": "#EF553B",
    "s2": "#00CC96",
    "s3": "#AB63FA",
    "s4": "#FFA15A",
    "s5": "#19D3F3"
}



for p in rates:
    df1 = df.query('selling_back == False and rate == @p')
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # npv bar plot
    for scenario in df1['scenario'].unique():
        df2 = df1[df1['scenario'] == scenario]
        fig.add_trace(go.Bar(x=df2['house_number'], y=df2['npv'], name=f'npv-{scenario}', marker_color=color_dict[scenario]), secondary_y=False)

    # irr line plot
    for scenario in df1['scenario'].unique():
        df2 = df1[df1['scenario'] == scenario]
        fig.add_trace(go.Scatter(x=df2['house_number'], y=df2['irr'], mode='lines+markers', name=f'irr-{scenario}', line=dict(color=color_dict[scenario])), secondary_y=True)

    # Set titles and labels
    fig.update_layout(title_text=f"npv & irr, rate: {p}")
    fig.update_xaxes(title_text="house_number")
    fig.update_yaxes(title_text="npv", secondary_y=False)
    fig.update_yaxes(title_text="irr", secondary_y=True)
    
    # Show the figure
    fig.show()


### detailed results for one of the Houses

In [47]:
hn = 'House 4'

In [61]:
import plotly.graph_objects as go

# Define color dictionary
color_dict = {
    "s0": "#636EFA",
    "s1": "#EF553B",
    "s2": "#00CC96",
    "s3": "#AB63FA",
    "s4": "#FFA15A",
    "s5": "#19D3F3"
}

legend = {
    "s0": "s0: SQ",
    "s1": "s1: Bat",
    "s2": "s2: Bat+PV",
    "s3": "s3: EV",
    "s4": "s4: EV+PV",
    "s5": "s5: Bat+EV+PV"
}


df1 = df.query('agent == "PRO" and selling_back == True and house_number == @hn')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# npv bar plot
for scenario in df1['scenario'].unique():
    df2 = df1[df1['scenario'] == scenario]
    fig.add_trace(go.Bar(x=df2['rate'], y=df2['npv'], name=f'npv-{legend[scenario]}', marker_color=color_dict[scenario]), secondary_y=False)

# irr line plot
for scenario in df1['scenario'].unique():
    df2 = df1[df1['scenario'] == scenario]
    fig.add_trace(go.Scatter(x=df2['rate'], y=df2['irr'], mode='lines+markers', name=f'irr-{legend[scenario]}', line=dict(color=color_dict[scenario])), secondary_y=True)

# Set titles and labels
# fig.update_layout(title_text=f"npv & irr, {hn} (selling back is allowed)")
fig.update_xaxes(title_text=f'rates (selling back is allowed)')
fig.update_yaxes(title_text="npv", secondary_y=False)
fig.update_yaxes(title_text="irr", secondary_y=True)

# Show the figure
fig.show()


df1 = df.query('agent == "PRO" and selling_back == False and house_number ==  @hn')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# npv bar plot
for scenario in df1['scenario'].unique():
    df2 = df1[df1['scenario'] == scenario]
    fig.add_trace(go.Bar(x=df2['rate'], y=df2['npv'], name=f'npv-{legend[scenario]}', marker_color=color_dict[scenario]), secondary_y=False)

# irr line plot
for scenario in df1['scenario'].unique():
    df2 = df1[df1['scenario'] == scenario]
    fig.add_trace(go.Scatter(x=df2['rate'], y=df2['irr'], mode='lines+markers', name=f'irr-{legend[scenario]}', line=dict(color=color_dict[scenario])), secondary_y=True)

# Set titles and labels
# fig.update_layout(title_text=f"npv & irr, {hn} (selling back is NOT allowed)")
fig.update_xaxes(title_text=f'rates (selling back is NOT allowed)')
fig.update_yaxes(title_text="npv", secondary_y=False)
fig.update_yaxes(title_text="irr", secondary_y=True)

# Show the figure
fig.show()



color_dict = {
    "s0": "#636EFA",
    "s1": "#EF553B",
    "s2": "#00CC96",
    "s3": "#AB63FA",
    "s4": "#FFA15A",
    "s5": "#19D3F3"
}

# for house in unique_house_numbers:
for house in [hn]:
    df_subset = df[df['house_number'] == house].query('agent == "PRO" and selling_back == True')
    
    fig = go.Figure()

    for scenario in df_subset['scenario'].unique():
        df_scenario = df_subset[df_subset['scenario'] == scenario]
        fig.add_trace(go.Bar(
            x=df_scenario['rate'],
            y=df_scenario['network_cost'],
            name=legend[scenario],
            marker_color=color_dict[scenario]
        ))

    fig.update_layout(
        # title_text='Network Cost for ' + house + '(when selling back is allowed)',
        xaxis_title=f'rates (selling back is allowed)',
        yaxis_title='Network Cost'
    )
    pk = df_subset.query('rate == "Time-of-Use EV & flat" and scenario == "s4"').peaks.values[0]
    pkt = [float(i) for i in pk[1:-1].split()]
    fig.show()
for house in [hn]:
    df_subset = df[df['house_number'] == house].query('agent == "PRO" and selling_back == False')
    
    fig = go.Figure()

    for scenario in df_subset['scenario'].unique():
        df_scenario = df_subset[df_subset['scenario'] == scenario]
        fig.add_trace(go.Bar(
            x=df_scenario['rate'],
            y=df_scenario['network_cost'],
            name=legend[scenario],
            marker_color=color_dict[scenario]
        ))

    fig.update_layout(
        # title_text='Network Cost for ' + house + '(when selling back is Not allowed)',
        xaxis_title=f'rates (selling back is NOT allowed)',
        yaxis_title='Network Cost'
    )
    pk = df_subset.query('rate == "Time-of-Use EV & flat" and scenario == "s4"').peaks.values[0]
    pkf = [float(i) for i in pk[1:-1].split()]

    fig.show()


In [120]:
# Define color dictionary and legend
color_dict = {
    "s0": "#636EFA",
    "s1": "#EF553B",
    "s2": "#00CC96",
    "s3": "#AB63FA",
    "s4": "#FFA15A",
    "s5": "#19D3F3"
}

legend = {
    "s0": "s0: SQ",
    "s1": "s1: Bat",
    "s2": "s2: Bat+PV",
    "s3": "s3: EV",
    "s4": "s4: EV+PV",
    "s5": "s5: Bat+EV+PV"
}

# Extract data for plotting
x_axis = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

for p in ['Time-of-Use', 'Time-of-Use EV & flat']:
    dff = df.query("rate == @p and house_number == 'House 4' and selling_back == True")
    peaks_data = df['peaks'].apply(lambda x: [float(val) for val in x.strip("[]").split()]).tolist()
    scenarios = dff['scenario'].tolist()

    # Create a figure
    fig = go.Figure()

    # Iterate over each scenario and add trace
    for i, scenario in enumerate(scenarios):
        peaks = [4*float(value) for value in dff.peaks.to_numpy()[i].strip('[]').split()]
        color = color_dict[scenario]
        label = legend[scenario]
        
        fig.add_trace(
            go.Scatter(
                x=x_axis,
                y=peaks,
                mode='lines+markers',
                name=label,
                line=dict(color=color, width=2),
                marker=dict(color=color, size=8),
            )
        )

    # Set layout
    fig.update_layout(
        # title="Peak Loads for Different Scenarios",
        xaxis_title=f"Months (rate: {p})",
        yaxis_title="Peak Load (kW)",
        legend_title=f"Scenarios",
        yaxis=dict(range=[5, 25])
    )

    # Show the plot
    fig.show()

### sensitivity analysis on load pattern

In [146]:
df1 = df.query("house_number == 'House 1' and selling_back == True")[['rate','scenario','npv']].pivot(index='rate', columns='scenario', values='npv')
df2 = df.query("house_number == 'House 2' and selling_back == True")[['rate','scenario','npv']].pivot(index='rate', columns='scenario', values='npv')
df3 = df.query("house_number == 'House 3' and selling_back == True")[['rate','scenario','npv']].pivot(index='rate', columns='scenario', values='npv')
df4 = df.query("house_number == 'House 4' and selling_back == True")[['rate','scenario','npv']].pivot(index='rate', columns='scenario', values='npv')
df_combined = pd.concat([df1, df2, df3, df4]).assign(House=['House 1'] * len(df1) + ['House 2'] * len(df2) + ['House 3'] * len(df3) + ['House 4'] * len(df4))
print(df_combined.round())

scenario                s0        s1        s2       s3       s4        s5    House
rate                                                                               
RTLMP                 0.00 -24493.00 -30137.00  1623.00 -8406.00 -14516.00  House 1
TOU&RTLMP             0.00 -24109.00 -23946.00 -1258.00 -7490.00 -11331.00  House 1
Time-of-Use           0.00 -12819.00  -6279.00  6671.00 11221.00   9712.00  House 1
Time-of-Use EV & flat 0.00 -27300.00 -16455.00  6413.00 11938.00   5359.00  House 1
flat                  0.00 -27300.00 -16468.00 -4549.00   964.00  -5615.00  House 1
gmp                   0.00 -15953.00 -19769.00  1745.00 -6458.00  -5134.00  House 1
RTLMP                 0.00 -24493.00 -30137.00  1623.00 -8407.00 -14517.00  House 2
TOU&RTLMP             0.00 -24305.00 -24016.00  -764.00 -7645.00 -11808.00  House 2
Time-of-Use           0.00 -12820.00  -6279.00  6672.00 11221.00   9711.00  House 2
Time-of-Use EV & flat 0.00 -27300.00 -16455.00  6413.00 11938.00   5359.00  

sensitivity analysis on battery capacity (0.33, 0.67, 1, 1.33, 1.67, 2)

In [13]:
hn = 'House 4'

In [14]:
data = {'agent': [], 'house_number' : [], 'battery_cap_coeff': [], 'scenario': [], 'rate': [], 'selling_back': [], 'annual_cost': [], 'saving': [], 'gmp_trade_cost': [], 'network_cost': [], 'npv': [], 'irr': [], 'load_from_grid': [], 'load_to_grid': [], 'peaks': [], 'investment_cost': []}
df_s1 = pd.DataFrame(data)


# N: life time 
N= 15
# ir: interest rate
ir = 0.0619
cap = [0.33, 0.67, 1, 1.33, 1.67, 2]
scenarios = ['s1', 's2', 's5']
rates = ['flat', 'Time-of-Use', 'Time-of-Use EV & flat', 'RTLMP', 'TOU&RTLMP']
sell = True

total_iterations = len(cap) *  len(rates)*  len(scenarios)
progress_bar = tqdm(total=total_iterations, desc="Processing", ncols=100)
for s in scenarios:
    for c in cap:
        with open('db.pkl', 'rb') as file:
            db = pickle.load(file)
        db['scenarios'][s]['B_max'] *= c
        if s in ['s1', 's2']:
            db['scenarios'][s]['price'] += 9100 * (c-1)/0.3333
        elif s == 's5':
            db['scenarios'][s]['price'] += 6579 * (c-1)/0.3333 
        db['scenarios'][s]['U_c'] = db['scenarios'][s]['B_max']/10.8
        db['scenarios'][s]['U_d'] = db['scenarios'][s]['B_max']/10.8
        for p in rates:
            new_row = {'house_number': hn, 'battery_cap_coeff': str(c), 'scenario': s, 'rate': p, 'selling_back': sell}
            new_row['agent'] = 'GMP' if p == 'gmp' else 'PRO'
            objval, peaks, gmp_trade_revenue, from_grid, to_grid = model(db=db, hn=hn, s=s, p=p, sell=True)
            new_row['network_cost'] = (1.35*5+9) * 4 * np.sum(peaks)
            new_row['annual_cost'] = objval
            new_row['load_from_grid'] = from_grid
            new_row['load_to_grid'] = to_grid
            new_row['gmp_trade_cost'] = gmp_trade_revenue
            new_row['peaks'] = str(np.round(peaks,2))
            new_row['investment_cost'] = db['scenarios'][s]['price']
            if s == 's0':
                saving = 0
            else:
                cc = str(c)
                sq_cost = df.query("house_number == @hn and rate == @p and scenario == 's0' and selling_back == True")['annual_cost'].values[0]
                saving = sq_cost - objval
            new_row['saving'] = saving
            returns = [-db['scenarios'][s]['price']]+N*[saving]
            new_row['npv'] = npf.npv(ir, returns)
            new_row['irr'] = 100 * npf.irr(returns)                
            new_row_df = pd.DataFrame([new_row])
            df_s1 = pd.concat([df_s1, new_row_df], ignore_index=True)
            progress_bar.update(1)
df_s1_backup = df_s1.copy()
progress_bar.close()
db['df_s1'] = df_s1

Processing: 100%|███████████████████████████████████████████████████| 90/90 [50:57<00:00, 33.97s/it]


In [19]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Scenarios
scenarios = ['s1', 's2', 's5']

# Get the unique battery_cap_coeff values and assign a color to each
battery_cap_coeffs = df_s1['battery_cap_coeff'].unique()
num_colors = len(px.colors.qualitative.Plotly)
color_dict = {battery_cap_coeffs[i]: px.colors.qualitative.Plotly[i % num_colors] for i in range(len(battery_cap_coeffs))}


for s in scenarios:
    dff = df_s1.query('scenario == @s')

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # npv bar plot and irr line plot
    for battery_cap_coeff in dff['battery_cap_coeff'].unique():
        df2 = dff[dff['battery_cap_coeff'] == battery_cap_coeff]
        fig.add_trace(go.Bar(x=df2['rate'], y=df2['npv'], name=f'npv-{battery_cap_coeff}', marker_color=color_dict[battery_cap_coeff]), secondary_y=False)
        fig.add_trace(go.Scatter(x=df2['rate'], y=df2['irr'], mode='lines+markers', name=f'irr-{battery_cap_coeff}', line=dict(color=color_dict[battery_cap_coeff])), secondary_y=True)

    # Set titles and labels
    # fig.update_layout(title_text=f"npv & irr")
    fig.update_xaxes(title_text=f"rate (scenario: {s})")
    fig.update_yaxes(title_text="npv", secondary_y=False)
    fig.update_yaxes(title_text="irr", secondary_y=True)
    
    # Show the figure
    fig.show()


sensitivity analysis on solar panel capacity (0.33, 0.67, 1, 1.33, 1.67, 2)

In [16]:
data = {'agent': [], 'house_number' : [], 'pv_cap_coeff': [], 'scenario': [], 'rate': [], 'selling_back': [], 'annual_cost': [], 'saving': [], 'gmp_trade_cost': [], 'network_cost': [], 'npv': [], 'irr': [], 'load_from_grid': [], 'load_to_grid': [], 'peaks': [], 'investment_cost': []}
df_s2 = pd.DataFrame(data)


# N: life time 
N= 15
# ir: interest rate
ir = 0.0619
cap = [0.2, 0.4, 0.6, 0.8, 1]
scenarios = ['s2', 's4', 's5']
rates = ['flat', 'Time-of-Use', 'Time-of-Use EV & flat', 'RTLMP', 'TOU&RTLMP']
sell = True

total_iterations = len(cap) *  len(rates)*  len(scenarios)
progress_bar = tqdm(total=total_iterations, desc="Processing", ncols=100)
for s in scenarios:
    for c in cap:
        with open('db.pkl', 'rb') as file:
            db = pickle.load(file)
        db['ld'].PV *= c
        db['scenarios'][s]['price'] += 13145 * (c-1)
        for p in rates:
            new_row = {'house_number': hn, 'pv_cap_coeff': str(c), 'scenario': s, 'rate': p, 'selling_back': sell}
            new_row['agent'] = 'GMP' if p == 'gmp' else 'PRO'
            objval, peaks, gmp_trade_revenue, from_grid, to_grid = model(db=db, hn=hn, s=s, p=p, sell=True)
            new_row['network_cost'] = (1.35*5+9) * 4 * np.sum(peaks)
            new_row['annual_cost'] = objval
            new_row['load_from_grid'] = from_grid
            new_row['load_to_grid'] = to_grid
            new_row['gmp_trade_cost'] = gmp_trade_revenue
            new_row['peaks'] = str(np.round(peaks,2))
            new_row['investment_cost'] = db['scenarios'][s]['price']
            if s == 's0':
                saving = 0
            else:
                cc = str(c)
                sq_cost = df.query("house_number == @hn and rate == @p and scenario == 's0' and selling_back == True")['annual_cost'].values[0]
                saving = sq_cost - objval
            new_row['saving'] = saving
            returns = [-db['scenarios'][s]['price']]+N*[saving]
            new_row['npv'] = npf.npv(ir, returns)
            new_row['irr'] = 100 * npf.irr(returns)                
            new_row_df = pd.DataFrame([new_row])
            df_s2 = pd.concat([df_s2, new_row_df], ignore_index=True)
            progress_bar.update(1)
progress_bar.close()
df_s2_backup = df_s2.copy()
db['df_s2'] = df_s2

Processing: 100%|███████████████████████████████████████████████████| 75/75 [27:37<00:00, 22.09s/it]


In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Scenarios
scenarios = ['s2', 's4', 's5']

# Get the unique pv_cap_coeff values and assign a color to each
pv_cap_coeffs = df_s2['pv_cap_coeff'].unique()
pv_cap_coeffs_2 = pv_cap_coeffs*10
num_colors = len(px.colors.qualitative.Plotly)
color_dict = {pv_cap_coeffs[i]: px.colors.qualitative.Plotly[i % num_colors] for i in range(len(pv_cap_coeffs))}


for s in scenarios:
    dff = df_s2.query('scenario == @s')

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # npv bar plot and irr line plot
    for pv_cap_coeff in dff['pv_cap_coeff'].unique():
        df2 = dff[dff['pv_cap_coeff'] == pv_cap_coeff]
        fig.add_trace(go.Bar(x=df2['rate'], y=df2['npv'], name=f'npv-{int(10*float(pv_cap_coeff))}', marker_color=color_dict[pv_cap_coeff]), secondary_y=False)
        fig.add_trace(go.Scatter(x=df2['rate'], y=df2['irr'], mode='lines+markers', name=f'irr-{int(10*float(pv_cap_coeff))}', line=dict(color=color_dict[pv_cap_coeff])), secondary_y=True)

    # Set titles and labels
    # fig.update_layout(title_text=f"npv & irr, scenario: {s}")
    fig.update_xaxes(title_text=f"rate (scenario: {s})")
    fig.update_yaxes(title_text="npv", secondary_y=False)
    fig.update_yaxes(title_text="irr", secondary_y=True)
    
    # Show the figure
    fig.show()


In [39]:
with open('db.pkl', 'wb') as file:
    pickle.dump(db, file, protocol=pickle.HIGHEST_PROTOCOL)